# Training Graph Neural Networks via Self-Supervised Learning: Experiments and Analysis

Latest Update: June 6th, 2022

## Install Pytorch Geometric

In [1]:
# !pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
# !pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
# !pip install -q torch-geometric

In [2]:
import torch
import random
import numpy as np
import os
import math
import networkx as nx
import shutil
from itertools import repeat
from copy import deepcopy
from datetime import datetime
import time
import matplotlib.pyplot as plt

In [3]:
print("Pytorch Version:", torch.__version__)
print("CUDA Available?", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)
print("+-----------------------------------------------------------------------------+")
!nvcc --version
print("+-----------------------------------------------------------------------------+")
!nvidia-smi

Pytorch Version: 1.9.1+cu111
CUDA Available? True
CUDA Version: 11.1
+-----------------------------------------------------------------------------+
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
+-----------------------------------------------------------------------------+
Fri Dec 17 15:30:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 470.74       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+=================

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
path = "./Datasets/"
print(path)

./Datasets/


In [6]:
# dataset_name = "MUTAG"
# dataset_name = "PROTEINS"
# dataset_name = "NCI1"

dataset_name = "DD"

In [7]:
now = datetime.now()
dt = now.strftime("%Y%m%d-%H%M%S")
log_path = "./Logs09/logs_"+dataset_name+"_"+dt+"/"
os.mkdir(log_path)
print(log_path)

./Logs09/logs_DD_20211217-153041/


## Seed & Plot Function

In [8]:
def my_seed(seed):
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)

my_seed(46)    

In [10]:
from torch_geometric.utils.convert import to_networkx

def draw_graph(pygeo_graph):
    G = to_networkx(pygeo_graph)
    pos = nx.circular_layout(G)
    nx.draw(G, pos, with_labels=True, node_color="pink", node_size=200)
    plt.show()

def plot_data(path, data, title_str, save_file=False):
    now = datetime.now()
    dt = now.strftime("%Y%m%d-%H%M%S")
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(data)+1), data)
    plt.title(title_str) # title
    plt.xlabel("Epochs") # x label
    plt.xticks(range(0, len(data)+1, int(len(data)/10)))
    if save_file == True:
      plt.savefig(path+dt+"_"+title_str+".png")
      print("Save!")
    plt.show()

def record__to_txt(path, data, filename):
    now = datetime.now()
    dt = now.strftime("%Y%m%d-%H%M%S")
    with open(path+dt+"_"+filename+".txt", "w") as f:
        for item in data:
            f.write("%s\n" % item)    

In [11]:
from datetime import datetime, timezone, timedelta

tz = timezone(timedelta(hours=+8))
now = datetime.now(tz)
dt = now.strftime("%Y-%m-%d %H:%M:%S")

200

## Dataset



### TUDataset

- TU Datasets: https://chrsmrrs.github.io/datasets/docs/datasets/
- Ref: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/datasets/tu_dataset.html
- Open Graph Benchmark: https://ogb.stanford.edu/


In [12]:
!pip list

Package                           Version
--------------------------------- ---------------
absl-py                           0.13.0
argon2-cffi                       20.1.0
asn1crypto                        0.24.0
astunparse                        1.6.3
async-generator                   1.10
attrs                             21.2.0
backcall                          0.2.0
backports.entry-points-selectable 1.1.0
bleach                            4.0.0
cached-property                   1.5.2
cachetools                        4.2.2
certifi                           2021.5.30
cffi                              1.14.6
charset-normalizer                2.0.4
clang                             5.0
cryptography                      2.1.4
cycler                            0.10.0
dataclasses                       0.8
decorator                         4.4.2
defusedxml                        0.7.1
distlib                           0.3.3
entrypoints                       0.3
filelock                 

In [13]:
# !pip install --upgrade torch torchvision

In [14]:
from torch_geometric.data import InMemoryDataset, download_url, extract_zip, DataLoader, Data
from torch_geometric.datasets import TUDataset
from torch_geometric.io import read_tu_data

from typing import Optional, Callable, List
import copy

from torch_geometric.data.collate import collate
from torch_geometric.data.separate import separate
from torch_geometric.data.dataset import Dataset, IndexType

import os
import os.path as osp

class TUDataset_AUG(InMemoryDataset):
    ## Ref: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/datasets/tu_dataset.html
    ## Ref: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/data/in_memory_dataset.html

    url = 'https://www.chrsmrrs.com/graphkerneldatasets'
    cleaned_url = ('https://raw.githubusercontent.com/nd7141/'
                    'graph_datasets/master/datasets')

    def __init__(self, root, name, transform=None, pre_transform=None,
            pre_filter=None, use_node_attr=False, use_edge_attr=False,
            cleaned=False, aug=None, proportion=[0.2, 0.2]):
        self.name = name
        self.cleaned = cleaned
        super(TUDataset_AUG, self).__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        self.aug = aug
        self.proportion = proportion

        if self.data.x is not None and not use_node_attr:
            num_node_attributes = self.num_node_attributes
            self.data.x = self.data.x[:, num_node_attributes:]
        if self.data.edge_attr is not None and not use_edge_attr:
            num_edge_attributes = self.num_edge_attributes
            self.data.edge_attr = self.data.edge_attr[:, num_edge_attributes:]

        # for those dataset does not have node attribute
        if not (self.name == "MUTAG" or self.name == "PTC_MR" or self.name == "DD" or self.name == "PROTEINS" or self.name == "NCI1" or self.name == "NCI109"):
            print("No Node Feature...處理中...")
            edge_index = self.data.edge_index[0, :].numpy()
            num_edge = self.data.edge_index.size()[1]
            nlist = [edge_index[n] + 1 for n in range(num_edge - 1) if edge_index[n] > edge_index[n + 1]]
            nlist.append(edge_index[-1] + 1)

            num_node = np.array(nlist).sum()          
            # if the node does note have attribute, it can fill with 0 or 1.
            # self.data.x = torch.zeros((num_node, 1))
            self.data.x = torch.ones((num_node, 1))

            edge_slice = [0]
            k = 0
            for n in nlist:
                k = k + n
                edge_slice.append(k)
            self.slices["x"] = torch.tensor(edge_slice)

    @property
    def raw_dir(self) -> str:
        name = f'raw{"_cleaned" if self.cleaned else ""}'
        return osp.join(self.root, self.name, name)

    @property
    def processed_dir(self) -> str:
        name = f'processed{"_cleaned" if self.cleaned else ""}'
        return osp.join(self.root, self.name, name)

    @property
    def num_node_labels(self):
        if self.data.x is None:
            return 0
        for i in range(self.data.x.size(1)):
            if self.data.x[:, i:].sum().item() == self.data.x.size(0):
                return self.data.x.size(1) - i
        return 0

    @property
    def num_node_attributes(self):
        if self.data.x is None:
            return 0
        return self.data.x.size(1) - self.num_node_labels

    @property
    def num_edge_labels(self) -> int:
        if self.data.edge_attr is None:
            return 0
        for i in range(self.data.edge_attr.size(1)):
            if self.data.edge_attr[:, i:].sum() == self.data.edge_attr.size(0):
                return self.data.edge_attr.size(1) - i
        return 0

    @property
    def num_edge_attributes(self) -> int:
        if self.data.edge_attr is None:
            return 0
        return self.data.edge_attr.size(1) - self.num_edge_labels

    @property
    def raw_file_names(self):
        names = ['A', 'graph_indicator']
        return ['{}_{}.txt'.format(self.name, name) for name in names]


    @property
    def processed_file_names(self):
        return 'data.pt'

    @property
    def num_node_features(self):
        r"""Returns the number of features per node in the dataset."""
        return self[0][0].num_node_features

    def download(self):
        url = self.cleaned_url if self.cleaned else self.url
        folder = osp.join(self.root, self.name)
        path = download_url(f'{url}/{self.name}.zip', folder)
        extract_zip(path, folder)
        os.unlink(path)
        shutil.rmtree(self.raw_dir)
        os.rename(osp.join(folder, self.name), self.raw_dir)

    def process(self):
        self.data, self.slices = read_tu_data(self.raw_dir, self.name)

        if self.pre_filter is not None:
            data_list = [self.get(idx) for idx in range(len(self))]
            data_list = [data for data in data_list if self.pre_filter(data)]
            self.data, self.slices = self.collate(data_list)

        if self.pre_transform is not None:
            data_list = [self.get(idx) for idx in range(len(self))]
            data_list = [self.pre_transform(data) for data in data_list]
            self.data, self.slices = self.collate(data_list)

        torch.save((self.data, self.slices), self.processed_paths[0])

    def __repr__(self) -> str:
        return f'{self.name}({len(self)})'


    def get(self, idx):
        data = self.data.__class__()


        if hasattr(self.data, '__num_nodes__'):
            data.num_nodes = self.data.__num_nodes__[idx]

        for key in self.data.keys:
            item, slices = self.data[key], self.slices[key]
            if torch.is_tensor(item):
                s = list(repeat(slice(None), item.dim()))
                s[self.data.__cat_dim__(key, item)] = slice(slices[idx], slices[idx + 1])
            else:
                s = slice(slices[idx], slices[idx + 1])
            data[key] = item[s]



        node_num = data.edge_index.max()
        sl = torch.tensor([[n,n] for n in range(node_num)]).transpose(0, 1)
        data.edge_index = torch.cat((data.edge_index, sl), dim=1)

        # print(self.proportion)
        # print(self.aug)
        
        prop1 = self.proportion[0]
        prop2 = self.proportion[1]

        aug1 = self.aug[0]
        aug2 = self.aug[1]

        ## AUGMENTATION ##
        if aug1 == "node_dropping":
            data_aug1 = Augmantations.node_dropping(deepcopy(data), prop1)
        elif aug1 == "edge_perturbation":
            data_aug1 = Augmantations.edge_perturbation(deepcopy(data), prop1)
        elif aug1 == "subgraph":
            data_aug1 = Augmantations.subgraph(deepcopy(data), prop1)
        elif aug1 == "attribute_masking":
            data_aug1 = Augmantations.attribute_masking(deepcopy(data), prop1)

        ## DO NOTHING ##
        elif aug1 == "raw":
            data_aug1 = deepcopy(data)

        else:
            print("Augmentation Error!")
            assert False

        ## AUGMENTATION ##
        if aug2 == "node_dropping":
            data_aug2 = Augmantations.node_dropping(deepcopy(data), prop2)
        elif aug2 == "edge_perturbation":
            data_aug2 = Augmantations.edge_perturbation(deepcopy(data), prop2)
        elif aug2 == "subgraph":
            data_aug2 = Augmantations.subgraph(deepcopy(data), prop2)
        elif aug2 == "attribute_masking":
            data_aug2 = Augmantations.attribute_masking(deepcopy(data), prop2)

        ## DO NOTHING ##
        elif aug2 == "raw":
            data_aug2 = deepcopy(data)

        else:
            print("Augmentation Error!")
            assert False

        return data_aug1, data_aug2
        # return: (original raw data, augmentation data)


In [17]:
# dataset_num_features

## Augmentation

- Ref: You, Y., Chen, T., Sui, Y., Chen, T., Wang, Z., & Shen, Y. (2020). Graph contrastive learning with augmentations. Advances in Neural Information Processing Systems, 33, 5812-5823.
- https://arxiv.org/pdf/2010.13902.pdf

### Node dropping 

Given the graph $\mathcal{G}$, node dropping will randomly discard certain portion of vertices along with their connections. The underlying prior enforced by it is that missing part of vertices does not affect the semantic meaning of $\mathcal{G}$. Each node’s dropping probability follows a default i.i.d.
uniform distribution (or any other distribution).

### Edge perturbation

It will perturb the connectivities in $\mathcal{G}$ through randomly adding or dropping certain ratio of edges. It implies that the semantic meaning of $\mathcal{G}$ has certain robustness to the edge connectivity pattern variances. We also follow an i.i.d. uniform distribution to add/drop each edge.

### Subgraph

This one samples a subgraph from $\mathcal{G}$ using random walk (the algorithm is summarized in Appendix A). It assumes that the semantics of $\mathcal{G}$ can be much preserved in its (partial) local structure.


### Attribute masking

Attribute masking prompts models to recover masked vertex attributes using their context information, i.e., the remaining attributes. The underlying assumption is that missing partial vertex attributes does not affect the model predictions much.


In [18]:
class Augmantations():
    def __init__(self, data, proportion=0.2):
        self.data = data
        self.proportion = proportion

    def node_dropping(data, proportion=0.2):
        node_num = data.x.size()[0]
        edge_num = data.edge_index.size()[1]
        drop_num = int(node_num * proportion) # how much to drop
        # print(node_num, edge_num, drop_num)
        idx_drop = np.random.choice(node_num, drop_num, replace=False) # dropped node
        edge_index = data.edge_index.numpy()
        adj = torch.zeros((node_num, node_num)) # adjacency matrix
        adj[edge_index[0], edge_index[1]] = 1
        adj[idx_drop, :] = adj[:, idx_drop] = 0 # dropped node = no edge link
        edge_index = adj.nonzero().transpose(0, 1) 
        data.edge_index = edge_index
        return data

    def edge_perturbation(data, proportion=0.2):
        node_num = data.x.size()[0]
        edge_num = data.edge_index.size()[1]
        drop_num = int(edge_num * proportion) # how much to perturbation
        # print(node_num, edge_num, drop_num)
        edge_index = data.edge_index.transpose(0, 1).numpy() # edge list, [start, end]
        edge_index = edge_index[np.random.choice(edge_num, edge_num-drop_num, replace=False)]
        data.edge_index = torch.tensor(edge_index).transpose(0, 1)
        return data 

    def subgraph(data, proportion=0.2):
        node_num = data.x.size()[0]
        edge_num = data.edge_index.size()[1]
        sub_num = int(node_num * proportion) 
        # print(node_num, edge_num, sub_num)
        edge_index = data.edge_index.numpy()
        idx_sub = [np.random.randint(node_num, size=1)[0]]
        idx_neighbor = set([n for n in edge_index[1][edge_index[0]==idx_sub[0]]])
        # print(idx_sub, idx_neighbor)
        count = 0
        while len(idx_sub) <= sub_num:
            count = count + 1
            if count > node_num or len(idx_neighbor) == 0:
                break # neighborhoods
            sample_node = np.random.choice(list(idx_neighbor))
            if sample_node in idx_sub:
                continue
            idx_sub.append(sample_node)
            idx_neighbor.union(set([n for n in edge_index[1][edge_index[0]==idx_sub[-1]]]))
        idx_drop = [n for n in range(node_num) if not n in idx_sub] # do not need
        idx_nondrop = idx_sub # keep in subgraph
        idx_dict = {idx_nondrop[n]:n for n in list(range(len(idx_nondrop)))}
        # print(idx_drop, idx_nondrop, idx_dict)
        edge_index = data.edge_index.numpy()
        adj = torch.zeros((node_num, node_num)) # adjacency matrix
        adj[edge_index[0], edge_index[1]] = 1
        adj[idx_drop, :] = adj[:, idx_drop] = 0 # dropped node = no edge link
        edge_index = adj.nonzero().transpose(0, 1) 
        data.edge_index = edge_index
        return data


    def attribute_masking(data, proportion=0.2):
        node_num, feat_dim = data.x.size()
        mask_num = int(node_num * proportion)
        # print(node_num, feat_dim, mask_num)
        idx_mask = np.random.choice(node_num, mask_num, replace=False)
        data.x[idx_mask] = torch.tensor(np.random.normal(loc=0.5, scale=0.5, size=(mask_num, feat_dim)), dtype=torch.float32)
        return data


## Models




In [19]:
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T 
from torch_geometric.nn import GINConv, global_add_pool

def D(p, z): 
    # negative cosine similarity
    # z = z.detach() # stop gradient
    # p = F.normalize(p, dim=1) # l2-normalize 
    # z = F.normalize(z, dim=1) # l2-normalize 
    # return -(p*z).sum(dim=1).mean()
    return -F.cosine_similarity(p, z.detach(), dim=-1).mean()  


### SimCLR


In [20]:
class SimCLR(nn.Module):
    def __init__(self, hidden_dim, enc_layer, proj_layer, dataset_num_features):
        super().__init__()
        self.embedding_dim = hidden_dim * enc_layer
        self.dataset_num_features = dataset_num_features
        self.encoder = Encoder(dataset_num_features, hidden_dim, enc_layer)
        # self.proj_layer = proj_layer
        ## Projection Head ##           
        self.projector = Projection_MLP(self.embedding_dim, self.embedding_dim, self.embedding_dim, proj_layer)

    def forward(self, x, edge_index, batch, num_graphs):
        y = self.encoder(x, edge_index, batch)        
        y = self.projector(y)                        
        return y        

    def loss(self, x_raw, x_aug):
        # NT-Xent Loss
        T = 0.2 # temperature setting (tau)

        batch_size = x_raw.size()[0]
        sim_matrix = torch.einsum("ik,jk->ij", x_raw, x_aug)

        x_raw_abs = x_raw.norm(dim=1)
        x_aug_abs = x_aug.norm(dim=1)

        sim_matrix = sim_matrix / torch.einsum("i,j->ij", x_raw_abs, x_aug_abs)
        sim_matrix = torch.exp(sim_matrix / T)
        pos_sim = torch.diagonal(sim_matrix)
        
        loss = pos_sim / (sim_matrix.sum(dim=1) - pos_sim)
        loss = -torch.log(loss).mean()
        # print("NT-Xent Loss:", loss.item())
        return loss        

### BarlowTwins

In [21]:
class BarlowTwins(nn.Module):
    def __init__(self, hidden_dim, enc_layer, proj_layer, dataset_num_features):
        super().__init__()
        self.embedding_dim = hidden_dim * enc_layer
        self.dataset_num_features = dataset_num_features
        # self.proj_layer = proj_layer
        self.encoder = Encoder(dataset_num_features, hidden_dim, enc_layer)      
        ## Projection Head ##           
        self.projector = Projection_MLP(self.embedding_dim, self.embedding_dim, self.embedding_dim, proj_layer)

    def forward(self, x, edge_index, batch, num_graphs):
        y = self.encoder(x, edge_index, batch)        
        y = self.projector(y)                        
        return y        

    def loss(self, x_raw, x_aug):
        ## Parameter ##
        scale_loss = 1/32
        lambd=3.9e-3

        batch_size, metric_dim = x_raw.size()

        ## Normalization 
        x_raw_norm = (x_raw - x_raw.mean(dim=0)) / x_raw.std(dim=0)  # [batch_size, metric_dim]
        x_aug_norm = (x_aug - x_aug.mean(dim=0)) / x_aug.std(dim=0)  # [batch_size, metric_dim]
        corr_matrix = (x_raw_norm.T @ x_aug_norm) / batch_size  # [metric_dim, metric_dim]

        on_diag = torch.diagonal(corr_matrix).add_(-1).pow(2).sum().mul(scale_loss)
        off_diag = corr_matrix.flatten()[:-1].view(metric_dim - 1, metric_dim + 1)[:, 1:].flatten()
        off_diag = off_diag.pow(2).sum().mul(scale_loss)

        loss = on_diag + (lambd * off_diag)
        # print("BarlowTwins Loss:", loss.item())
        return loss

### SimSiam

In [22]:
class Simsiam(nn.Module):
    def __init__(self, hidden_dim, enc_layer, proj_layer, dataset_num_features):
        super().__init__()
        self.embedding_dim = hidden_dim * enc_layer
        self.dataset_num_features = dataset_num_features
        # self.proj_layer = proj_layer
        self.encoder = Encoder(dataset_num_features, hidden_dim, enc_layer)      
        ## Projection Head ##           
        self.projector = Projection_MLP(self.embedding_dim, self.embedding_dim, self.embedding_dim, proj_layer)
        ## Predictor ##
        self.predictor = Prediction_MLP(self.embedding_dim, self.embedding_dim, self.embedding_dim)

    def forward(self, x, edge_index, batch, num_graphs):
        y = self.encoder(x, edge_index, batch)        
        y = self.projector(y)        
        return y        
    
    def loss(self, x_raw, x_aug):
        h = self.predictor

        z1, z2 = x_raw, x_aug # embedding: encoder input
        # print(x_raw.shape, x_aug.shape)

        p1, p2 = h(z1), h(z2) # predictor
        # print(p1.shape, p2.shape)

        loss = D(p1, z2) / 2 + D(p2, z1) / 2
        # print("Simsiam loss:", loss.item())
        return loss         

### Projection



In [23]:
class Projection_MLP(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, proj_layer):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(hidden_dim, out_dim),
            nn.BatchNorm1d(hidden_dim)
        )
        
        self.unilayer = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(inplace=True), 
            nn.Linear(hidden_dim, out_dim)
        )

        self.proj_layer = 3
    
    def set_layers(self, proj_layer):
        self.proj_layer = proj_layer

    def forward(self, x):
        if self.proj_layer == 3:
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
        elif self.proj_layer == 2:
            x = self.layer1(x)
            x = self.layer3(x)
        elif self.proj_layer == 1:
            x = self.unilayer(x)
        else:
            raise Exception
        return x 

### Predictor

Layer 1: Prediction MLP. The prediction MLP (h) has BN applied to its hidden fc layers. Its output fc does not have BN (ablation in Sec. 4.4) or ReLU. This MLP has 2 layers. The dimension of h’s input and output (z and p) is d = 2048, and h’s hidden layer’s dimension is 512, making h a bottleneck structure (ablation in supplement). 

Layer 2: Adding BN to the output of the prediction MLP h does not work well (Table 3d). We find that this is not about collapsing. The training is unstable and the loss oscillates.

Ref: https://github.com/PatrickHua/SimSiam

In [24]:
class Prediction_MLP(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim): # bottleneck structure
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True), # hidden layer
            nn.Linear(hidden_dim, out_dim) # output layer
        )

    def forward(self, x):
        x = self.layer(x)
        return x 

### Encoder

- GINConv
- Ref: https://arxiv.org/abs/1810.00826
- Xu et al.: How Powerful are Graph Neural Networks? (ICLR 2019) 
- https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GINConv


In [25]:
class Encoder(torch.nn.Module):
    def __init__(self, num_features, dim, enc_layer):
        super(Encoder, self).__init__()
        self.enc_layer = enc_layer
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()

        for i in range(enc_layer):     
            if i == 0:
                # initial layer
                # input: (num_feature -> dim)
                net = nn.Sequential(
                    nn.Linear(num_features, dim), 
                    nn.ReLU(), 
                    nn.Linear(dim, dim)
                    )     
            else:
                # connect layer after initail layer
                # input: (dim -> dim)
                net = nn.Sequential(
                    nn.Linear(dim, dim), 
                    nn.ReLU(), 
                    nn.Linear(dim, dim)
                    )              
            
            # conv_layers = GATConv()?                
            # https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GINConv
            # The graph isomorphism operator from the “How Powerful are Graph Neural Networks?” paper
            conv_layers = GINConv(net)
            batch_norm_layers = torch.nn.BatchNorm1d(dim)

            self.convs.append(conv_layers)
            self.bns.append(batch_norm_layers)

    def forward(self, x, edge_index, batch):
        if x is None:
            # fill with 0 or 1?
            x = torch.ones((batch.shape[0], 1)).to(device)
            # x = torch.zeros((batch.shape[0], 1)).to(device)

        x_ls = []
        for i in range(self.enc_layer):
            x = F.relu(self.convs[i](x, edge_index))
            x = self.bns[i](x)
            x_ls.append(x)

        # https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.glob.global_add_pool
        # Returns batch-wise graph-level-outputs by adding node features across the node dimension
        x_pool = [global_add_pool(x, batch) for x in x_ls]

        # concat embedding
        x = torch.cat(x_pool, dim=1)
        return x

    def get_embeddings(self, loader):
        print("Get Embedding...")
        emb_vector = []
        y_vector = [] # graph classification (target)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        with torch.no_grad():
            for data in loader:
                data_raw = data[0] # raw dara
                data_raw = data_raw.to(device)
                x, edge_index, batch = data_raw.x, data_raw.edge_index, data_raw.batch
                if x is None:
                    # fill with 0 or 1?
                    x = torch.ones((batch.shape[0],1))
                    # x = torch.zeros((batch.shape[0],1))
                    x = x.to(device)
                emb = self.forward(x, edge_index, batch)
                # print(type(emb), type(y))
                # print(type(emb), type(torch.from_numpy(y)))
                # y = data_raw.y
                # y = torch.from_numpy(y)
                emb_vector.append(emb.cpu().numpy())
                y_vector.append(data_raw.y.cpu().numpy())
        # concat                
        emb_vector = np.concatenate(emb_vector, axis=0)
        # emb_vector shape: hidden_dim * layer number
        y_vector = np.concatenate(y_vector, axis=0) 
        return emb_vector, y_vector

## Evaluate Embedding

Ref: InfoGrpah, https://github.com/fanyun-sun/InfoGraph/blob/master/unsupervised/evaluate_embedding.py

- SVC Classify
- Logistic Classify
- Random Forest Classify
- Linearsvc Classify


In [26]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.svm import SVC, LinearSVC

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import warnings


class LogReg(nn.Module):
    def __init__(self, ft_in, nb_classes):
        super(LogReg, self).__init__()
        self.fc = nn.Linear(ft_in, nb_classes)
        for m in self.modules():
            self.weights_init(m)

    def weights_init(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.fill_(0.0)

    def forward(self, seq):
        ret = self.fc(seq)
        return ret

def logistic_classify(x, y):
    nb_classes = np.unique(y).shape[0]
    xent = nn.CrossEntropyLoss()
    hid_units = x.shape[1]

    accs = []
    accs_val = [] #validation
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    for train_index, test_index in kf.split(x, y):
        train_embs, test_embs = x[train_index], x[test_index]
        train_lbls, test_lbls= y[train_index], y[test_index]

        train_embs, train_lbls = torch.from_numpy(train_embs).cuda(), torch.from_numpy(train_lbls).cuda()
        test_embs, test_lbls= torch.from_numpy(test_embs).cuda(), torch.from_numpy(test_lbls).cuda()

        log = LogReg(hid_units, nb_classes)
        log.cuda()
        opt = torch.optim.Adam(log.parameters(), lr=0.01, weight_decay=0.0)

        best_val = 0
        test_acc = None
        for it in range(100):
            log.train()
            opt.zero_grad()

            logits = log(train_embs)
            loss = xent(logits, train_lbls)

            loss.backward()
            opt.step()

        logits = log(test_embs)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == test_lbls).float() / test_lbls.shape[0]
        accs.append(acc.item())

        # Validation
        val_size = len(test_index)
        test_index = np.random.choice(test_index, val_size, replace=False).tolist()
        train_index = [i for i in train_index if not i in test_index]

        train_embs, test_embs = x[train_index], x[test_index]
        train_lbls, test_lbls= y[train_index], y[test_index]

        train_embs, train_lbls = torch.from_numpy(train_embs).cuda(), torch.from_numpy(train_lbls).cuda()
        test_embs, test_lbls= torch.from_numpy(test_embs).cuda(), torch.from_numpy(test_lbls).cuda()


        log = LogReg(hid_units, nb_classes)
        log.cuda()
        opt = torch.optim.Adam(log.parameters(), lr=0.01, weight_decay=0.0)

        best_val = 0
        test_acc = None
        for it in range(100):
            log.train()
            opt.zero_grad()

            logits = log(train_embs)
            loss = xent(logits, train_lbls)

            loss.backward()
            opt.step()

        logits = log(test_embs)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == test_lbls).float() / test_lbls.shape[0]
        accs_val.append(acc.item())
    return np.mean(accs), np.mean(accs_val)

def svc_classify(x, y, search):
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    accuracies = []
    accuracies_val = []
    for train_index, test_index in kf.split(x, y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)
        if search:
            params = {'C':[0.001, 0.01,0.1,1,10,100,1000]}
            classifier = GridSearchCV(SVC(), params, cv=5, scoring='accuracy', verbose=0)
        else:
            classifier = SVC(C=10)
        classifier.fit(x_train, y_train)
        accuracies.append(accuracy_score(y_test, classifier.predict(x_test)))

        # validation
        val_size = len(test_index)
        test_index = np.random.choice(train_index, val_size, replace=False).tolist()
        train_index = [i for i in train_index if not i in test_index]

        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)
        if search:
            params = {'C':[0.001, 0.01,0.1,1,10,100,1000]}
            classifier = GridSearchCV(SVC(), params, cv=5, scoring='accuracy', verbose=0)
        else:
            classifier = SVC(C=10)
        classifier.fit(x_train, y_train)
        accuracies_val.append(accuracy_score(y_test, classifier.predict(x_test)))
    return np.mean(accuracies), np.mean(accuracies_val)

def randomforest_classify(x, y, search):
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    accuracies = []
    accuracies_val = []
    for train_index, test_index in kf.split(x, y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        if search:
            params = {'n_estimators': [100, 200, 500, 1000]}
            classifier = GridSearchCV(RandomForestClassifier(), params, cv=5, scoring='accuracy', verbose=0)
        else:
            classifier = RandomForestClassifier()
        classifier.fit(x_train, y_train)
        accuracies.append(accuracy_score(y_test, classifier.predict(x_test)))

        # val
        val_size = len(test_index)
        test_index = np.random.choice(test_index, val_size, replace=False).tolist()
        train_index = [i for i in train_index if not i in test_index]

        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        if search:
            params = {'n_estimators': [100, 200, 500, 1000]}
            classifier = GridSearchCV(RandomForestClassifier(), params, cv=5, scoring='accuracy', verbose=0)
        else:
            classifier = RandomForestClassifier()
        classifier.fit(x_train, y_train)
        accuracies_val.append(accuracy_score(y_test, classifier.predict(x_test)))    
    return np.mean(accuracies), np.mean(accuracies_val)

def linearsvc_classify(x, y, search):
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    accuracies = []
    accuracies_val = []
    for train_index, test_index in kf.split(x, y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        if search:
            params = {'C':[0.001, 0.01,0.1,1,10,100,1000]}
            classifier = GridSearchCV(LinearSVC(), params, cv=5, scoring='accuracy', verbose=0)
        else:
            classifier = LinearSVC(C=10)
        classifier.fit(x_train, y_train)
        accuracies.append(accuracy_score(y_test, classifier.predict(x_test)))

        # val
        val_size = len(test_index)
        test_index = np.random.choice(train_index, val_size, replace=False).tolist()
        train_index = [i for i in train_index if not i in test_index]

        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        if search:
            params = {'C':[0.001, 0.01,0.1,1,10,100,1000]}
            classifier = GridSearchCV(LinearSVC(), params, cv=5, scoring='accuracy', verbose=0)
        else:
            classifier = LinearSVC(C=10)
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            classifier.fit(x_train, y_train)
        accuracies_val.append(accuracy_score(y_test, classifier.predict(x_test)))
    return np.mean(accuracies), np.mean(accuracies_val)

@ignore_warnings(category=ConvergenceWarning)
def evaluate_embedding(embeddings, labels, search=True):

    labels = preprocessing.LabelEncoder().fit_transform(labels)
    x, y = np.array(embeddings), np.array(labels)
    print(x.shape, y.shape)

    # print("+-----------------------------------------------------------------------------+")
    # print("[Acc, Validation Acc]")
    # logreg_acc, logreg_acc_val = logistic_classify(x, y)
    # log_result = [np.mean(logreg_acc), np.mean(logreg_acc_val)]
    # print('LogReg Accuracy:', log_result)

    svc_acc, svc_acc_val = svc_classify(x, y, search)
    scv_result = [np.mean(svc_acc), np.mean(svc_acc_val)]
    print("SVC Accuracy:", scv_result)

    # linsvc_acc, linsvc_acc_val = linearsvc_classify(x, y, search)
    # linsvc_result = [np.mean(linsvc_acc), np.mean(linsvc_acc_val)]
    # print('LinearSVC Accuracy:', linsvc_result)

    # rf_acc, rf_acc_val = randomforest_classify(x, y, search)
    # rf_result = [np.mean(rf_acc), np.mean(rf_acc_val)]
    # print('RandomForest Accuracy:', rf_result)
    # print("+-----------------------------------------------------------------------------+")

    # return log_result, scv_result, linsvc_result, rf_result
    return scv_result


## Train

### Arguments 



In [29]:
model_ls = ["SimCLR", "BarlowTwins", "Simsiam"]
aug_ls = ["node_dropping", "edge_perturbation", "subgraph", "attribute_masking"]
batch_size_ls = [64, 128]
hid_dim_ls = [64, 512]
enc_layers = [1, 2, 3]
proj_layer = [1, 2, 3]
lr = 0.001
epochs = 200
data_prop = 0.9 # use 90\% samples for self-learning training, 10 \% for supervised validation

### Training

Loss:
1. SimCLR (NX-Tent Loss)
2. Barlow Twins
3. Simsiam

- Self-supervised: large, unlabeled, augmentation
- supervised (eval): small, labeled, raw

In [31]:
with open(log_path+"log_"+dataset_name+".txt", mode="w+") as f:  
    f.write("{},{},{},{},{},{},{},{},{},{},{},{}".format("idx","data_prop","model_name", "augmentation", "batch_size", "enc_layer", "hidden_dim", "acc", "val_acc", "cost_time(sec)", "record_time", "emb_time"))
    f.write("\n")

In [32]:
dataset_tmp = TUDataset_AUG(path, name=dataset_name, aug=["raw", "raw"], proportion=[1,1])
data_size = math.floor(len(dataset_tmp) * data_prop)
train_idx = np.random.choice(range(len(dataset_tmp)), data_size, replace=False).tolist()
print(data_size)
print(train_idx)

1060
[553, 217, 346, 850, 1139, 72, 280, 861, 916, 589, 50, 182, 307, 1067, 172, 583, 831, 791, 1014, 246, 18, 1013, 59, 1023, 984, 69, 755, 65, 71, 742, 1153, 256, 856, 266, 427, 956, 679, 826, 541, 147, 910, 0, 922, 683, 131, 103, 91, 1001, 166, 558, 796, 1078, 811, 852, 351, 1002, 383, 145, 197, 1072, 789, 17, 606, 9, 608, 1111, 664, 414, 154, 290, 954, 667, 959, 1097, 652, 339, 184, 368, 327, 506, 933, 370, 36, 768, 67, 1158, 93, 1123, 47, 624, 311, 883, 484, 702, 379, 316, 794, 822, 818, 55, 375, 1008, 605, 584, 920, 575, 986, 675, 82, 468, 1104, 202, 428, 566, 85, 550, 914, 1142, 545, 1145, 870, 1149, 332, 732, 185, 401, 359, 540, 714, 277, 785, 56, 45, 330, 1137, 564, 941, 703, 157, 1056, 1025, 325, 394, 360, 1077, 611, 1058, 1131, 830, 1081, 857, 1105, 364, 57, 233, 291, 1065, 926, 181, 231, 230, 808, 731, 678, 588, 929, 226, 537, 680, 38, 746, 595, 733, 244, 138, 1039, 441, 216, 762, 220, 284, 570, 1166, 1103, 117, 476, 261, 312, 232, 424, 149, 1110, 183, 651, 776, 522, 898, 3

In [33]:
# dataset_aug = TUDataset_AUG(path, name=dataset_name, aug=["raw", "subgraph"], proportion=0.1)
# dataset_eval = TUDataset_AUG(path, name=dataset_name, aug=["raw", "raw"])

# dataset_num_features = dataset_eval[0][0].num_node_features if dataset_eval[0][0].num_node_features is not None else 1


In [34]:
# dataset_eval[0][0].num_node_features

In [35]:
def main_train(idx, train_idx, augmentation, batch_size, hidden_dim, enc_layer, proj_layer, model_name):    
    aug_method = ["raw", augmentation]
    proportion = [0.3, 0.3]
  
    dataset_aug = TUDataset_AUG(path, name=dataset_name, aug=aug_method, proportion=proportion)
    dataset_eval = TUDataset_AUG(path, name=dataset_name, aug=["raw", "raw"])
    print("!!!") # check features
    dataset_num_features = dataset_eval[0][0].num_node_features if dataset_eval[0][0].num_node_features is not None else 1
    print(dataset_num_features)
    print("!!!")
    

    test_idx = [i for i in range(len(dataset_aug)) if not i in train_idx]

    dataset_train, dataset_test = dataset_aug[train_idx], dataset_eval[test_idx]
    dataloader_train = DataLoader(dataset_train, batch_size=batch_size)  
    dataloader_test = DataLoader(dataset_test, batch_size=batch_size)

    print(train_idx)
    print("Number of Graphs (dataset_train):", len(dataset_train)) # 有幾張圖
    print("Number of Graphs (dataset_test):", len(dataset_test)) # 有幾張圖

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    if model_name == "SimCLR":
        model = SimCLR(hidden_dim, enc_layer, proj_layer, dataset_num_features).to(device)
    elif model_name == "BarlowTwins":
        model = BarlowTwins(hidden_dim, enc_layer, proj_layer, dataset_num_features).to(device)
    elif model_name == "Simsiam":
        model = Simsiam(hidden_dim, enc_layer, proj_layer, dataset_num_features).to(device)


    optimizer = torch.optim.Adam(model.parameters(), lr=lr)



    print("+-----------------------------------------------------------------------------+")
    print("Learning Rate:", lr)
    print("Epochs:", epochs)
    print("Batch Size:", batch_size)
    print("Hidden Dimension", hidden_dim)
    print("Number of Layer in Encoder:", enc_layer)
    print("Opimizer:", optimizer)
    print("+-----------------------------------------------------------------------------+")
    print("Device:", device)
    print("Model:")
    print(model)
    print("+-----------------------------------------------------------------------------+")


    model.train()

    losses = []
    start_time = time.time()
    print("Start Training...")
    print("+-----------------------------------------------------------------------------+")

    for epoch in range(1, epochs+1):
        loss_all = 0
        model.train()
        for data in dataloader_train:   
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

            data_raw, data_aug = data
            # raw data, augmentation data
            
            optimizer.zero_grad()
            data_raw = data_raw.to(device)
            x_raw = model(data_raw.x, data_raw.edge_index, data_raw.batch, data_raw.num_graphs)

            node_num = data_raw.x.size()[0] # number of nodes in this batch
            edge_idx = data_aug.edge_index.numpy()
            edge_num = edge_idx.shape[1] # number of edges in this batch
            idx_not_missing = [n for n in range(node_num) if (n in edge_idx[0] or n in edge_idx[1])]
            # via subgraph or node dropping, it might get isolated nodes.

            node_num_aug = len(idx_not_missing)
            data_aug.x = data_aug.x[idx_not_missing]      

            data_aug.batch = data_raw.batch[idx_not_missing]
            idx_dict = {idx_not_missing[n]:n for n in range(node_num_aug)}
            edge_idx = [[idx_dict[edge_idx[0, n]], idx_dict[edge_idx[1, n]]] for n in range(edge_num) if not edge_idx[0, n] == edge_idx[1, n]]
            data_aug.edge_index = torch.tensor(edge_idx).transpose(0, 1)
            ##

            data_aug = data_aug.to(device)
            x_aug = model(data_aug.x, data_aug.edge_index, data_aug.batch, data_aug.num_graphs)
            
            # print("x_raw:", x_raw.shape)
            # print("x_aug:", x_aug.shape)

            loss = model.loss(x_raw, x_aug)
            loss_all = loss_all + (loss.item() * data_raw.num_graphs)
            loss.backward()
            optimizer.step()
            
        print("Epoch", epoch, "| Loss =", loss_all/len(dataloader_train))
        losses.append(loss_all/len(dataloader_train))
    end_time = time.time()       
    ttl_time = end_time-start_time
    print("+-----------------------------------------------------------------------------+")
    print("Training Time:", ttl_time, "seconds.")

    record__to_txt(log_path, losses, dataset_name+"_Loss_"+str(idx))

    ## TEST

    model.eval() 

    start_time = time.time()       
    emb, y = model.encoder.get_embeddings(dataloader_test)
    # print(emb, y)
    # print("Embedding Shape:", emb.shape, y.shape)
    scv_result = evaluate_embedding(emb, y)

    end_time = time.time()       
    emb_time = end_time-start_time
    print("+-----------------------------------------------------------------------------+")
    print("Embedding Time:", emb_time, "seconds.")

    return scv_result[0], scv_result[1], ttl_time, emb_time

In [ ]:
idx = 0
for model_name in model_ls:
    for augmentation in aug_ls:
        for batch_size in batch_size_ls:
            for enc_layer in enc_layers:
                for hidden_dim in hid_dim_ls:
                    if idx<0:
                        pass
                    else:
                        print("+-----------------------------------------------------------------------------+")
                        print("Experient", idx)
                        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                        acc, val_acc, cost_time, emb_time = main_train(idx, train_idx, augmentation, batch_size, hidden_dim, enc_layer, proj_layer, model_name)
                        print("Experient", idx)

                        now = datetime.now()
                        dt = now.strftime("%Y%m%d-%H%M%S")
                        with open(log_path+"log_"+dataset_name+".txt", mode="a") as f:

                            f.write("{},{},{},{},{},{},{},{},{},{},{},{}".format(idx, data_prop, model_name, augmentation, batch_size, enc_layer, hidden_dim, acc, val_acc, cost_time, dt, emb_time))
                            f.write("\n")
                            print(dt, ":Log the record!")

                        if idx%1==0:
    #                         tz = timezone(timedelta(hours=+8))
    #                         now = datetime.now(tz)
    #                         dt = now.strftime("%Y-%m-%d %H:%M:%S")


                            msg = "[Sinica 0] "+ dataset_name + " | Exp " + str(idx) +" | " 
                            lineNotifyMessage(token, msg)

                    idx = idx + 1                        

+-----------------------------------------------------------------------------+
Experient 0
!!!
89
!!!
[553, 217, 346, 850, 1139, 72, 280, 861, 916, 589, 50, 182, 307, 1067, 172, 583, 831, 791, 1014, 246, 18, 1013, 59, 1023, 984, 69, 755, 65, 71, 742, 1153, 256, 856, 266, 427, 956, 679, 826, 541, 147, 910, 0, 922, 683, 131, 103, 91, 1001, 166, 558, 796, 1078, 811, 852, 351, 1002, 383, 145, 197, 1072, 789, 17, 606, 9, 608, 1111, 664, 414, 154, 290, 954, 667, 959, 1097, 652, 339, 184, 368, 327, 506, 933, 370, 36, 768, 67, 1158, 93, 1123, 47, 624, 311, 883, 484, 702, 379, 316, 794, 822, 818, 55, 375, 1008, 605, 584, 920, 575, 986, 675, 82, 468, 1104, 202, 428, 566, 85, 550, 914, 1142, 545, 1145, 870, 1149, 332, 732, 185, 401, 359, 540, 714, 277, 785, 56, 45, 330, 1137, 564, 941, 703, 157, 1056, 1025, 325, 394, 360, 1077, 611, 1058, 1131, 830, 1081, 857, 1105, 364, 57, 233, 291, 1065, 926, 181, 231, 230, 808, 731, 678, 588, 929, 226, 537, 680, 38, 746, 595, 733, 244, 138, 1039, 441, 216, 7

/usr/local/lib/python3.6/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


+-----------------------------------------------------------------------------+
Learning Rate: 0.001
Epochs: 200
Batch Size: 64
Hidden Dimension 64
Number of Layer in Encoder: 1
Opimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
+-----------------------------------------------------------------------------+
Device: cuda
Model:
SimCLR(
  (encoder): Encoder(
    (convs): ModuleList(
      (0): GINConv(nn=Sequential(
        (0): Linear(in_features=89, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
      ))
    )
    (bns): ModuleList(
      (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (projector): Projection_MLP(
    (layer1): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (

Epoch 172 | Loss = -16.751004191005933
Epoch 173 | Loss = -17.85443364872652
Epoch 174 | Loss = -16.79921214720782
Epoch 175 | Loss = -17.387923268710864
Epoch 176 | Loss = -17.270731617422665
Epoch 177 | Loss = -17.680277964648077
Epoch 178 | Loss = -17.7014180912691
Epoch 179 | Loss = -17.693357804242304
Epoch 180 | Loss = -17.262780834646787
Epoch 181 | Loss = -17.9941086769104
Epoch 182 | Loss = -17.821685875163357
Epoch 183 | Loss = -17.265101376701804
Epoch 184 | Loss = -17.396604481865378
Epoch 185 | Loss = -17.971850843990552
Epoch 186 | Loss = -18.96929754930384
Epoch 187 | Loss = -18.3002451728372
Epoch 188 | Loss = -18.193278733421774
Epoch 189 | Loss = -17.97198480718276
Epoch 190 | Loss = -18.47174302269431
Epoch 191 | Loss = -17.39201800963458
Epoch 192 | Loss = -18.82809075187234
Epoch 193 | Loss = -19.35253564049216
Epoch 194 | Loss = -18.337716467240277
Epoch 195 | Loss = -19.225754737854004
Epoch 196 | Loss = -18.659628251019647
Epoch 197 | Loss = -18.22701658922083
E

/usr/local/lib/python3.6/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


...

Epoch 1 | Loss = 242.7542112574858
Epoch 2 | Loss = 248.05309761271758
Epoch 3 | Loss = 250.84299704607795
Epoch 4 | Loss = 248.04947348201975
Epoch 5 | Loss = 244.91003917245303
Epoch 6 | Loss = 242.87850660436294
Epoch 7 | Loss = 242.85060231825884
Epoch 8 | Loss = 246.48118260327507
Epoch 9 | Loss = 243.8472624386058
Epoch 10 | Loss = 242.1112278769998
Epoch 11 | Loss = 245.20868918474983
Epoch 12 | Loss = 239.31035866456872
Epoch 13 | Loss = 244.46562407998476
Epoch 14 | Loss = 243.01919404198142
Epoch 15 | Loss = 234.43751537098603
Epoch 16 | Loss = 241.89988231658936
Epoch 17 | Loss = 242.08766129437615
Epoch 18 | Loss = 240.05304067275102
Epoch 19 | Loss = 240.9844233007992
Epoch 20 | Loss = 232.5845251644359
Epoch 21 | Loss = 236.73050386765425
Epoch 22 | Loss = 236.15784790936638
Epoch 23 | Loss = 232.58294481389663
Epoch 24 | Loss = 235.8281832302318
Epoch 25 | Loss = 239.44704207252053
Epoch 26 | Loss = 237.63546651952407
Epoch 27 | Loss = 233.92824313219856
Epoch 28 | Loss 

/usr/local/lib/python3.6/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1 | Loss = 249.48301292868223
Epoch 2 | Loss = 244.71859185835893
Epoch 3 | Loss = 244.77207026762122
Epoch 4 | Loss = 242.03783321380615
Epoch 5 | Loss = 242.90766631855683
Epoch 6 | Loss = 238.325932895436
Epoch 7 | Loss = 237.82067068885354
Epoch 8 | Loss = 242.8002835442038
Epoch 9 | Loss = 238.31339763192568
Epoch 10 | Loss = 241.1108351875754
Epoch 11 | Loss = 236.31346035003662
Epoch 12 | Loss = 238.04544723735137
Epoch 13 | Loss = 234.32114735771628
Epoch 14 | Loss = 238.0154987223008
Epoch 15 | Loss = 238.2483449823716
Epoch 16 | Loss = 236.53743210960837
Epoch 17 | Loss = 238.39635119718665
Epoch 18 | Loss = 235.34369193806367
Epoch 19 | Loss = 239.4831667507396
Epoch 20 | Loss = 236.22711551890654
Epoch 21 | Loss = 235.33859898062315
Epoch 22 | Loss = 237.19217732373406
Epoch 23 | Loss = 236.12289439930635
Epoch 24 | Loss = 235.14200479844038
Epoch 25 | Loss = 230.50735574610093
Epoch 26 | Loss = 233.8124855265898
Epoch 27 | Loss = 231.75601224338308
Epoch 28 | Loss = 

/usr/local/lib/python3.6/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1 | Loss = 254.45417813693777
Epoch 2 | Loss = 257.3828233270084
Epoch 3 | Loss = 255.51210050021902
Epoch 4 | Loss = 255.87279454399558
Epoch 5 | Loss = 260.4177159141092
Epoch 6 | Loss = 248.35234445684097
Epoch 7 | Loss = 254.48030477411606
Epoch 8 | Loss = 251.49771505243638
Epoch 9 | Loss = 252.49707827848547
Epoch 10 | Loss = 244.82803541071274
Epoch 11 | Loss = 247.8336627062629
Epoch 12 | Loss = 253.3652462678797
Epoch 13 | Loss = 254.14025906955496
Epoch 14 | Loss = 250.37598391140207
Epoch 15 | Loss = 250.87509811625762
Epoch 16 | Loss = 240.66060464522417


In [ ]:
dt = now.strftime("%Y%m%d-%H%M%S")

print("+-----------------------------------------------------------------------------+")                    
print("+-----------------------------------------------------------------------------+")
print("DONE!!!")
print(dt)
print("+-----------------------------------------------------------------------------+")
print("+-----------------------------------------------------------------------------+")

## Plot and Record